In [1]:
!pip install prophet
!pip install deep_translator

In [48]:
import pandas as pd
import numpy as np
from prophet import Prophet
from datetime import datetime
from sklearn.metrics import mean_squared_error
from deep_translator import GoogleTranslator
from IPython.display import display, HTML
pd.options.display.float_format = '{:.4f}'.format
pd.options.display.max_columns = None


In [3]:
# Pre-defined functions

def file_read(file_path):
    df = pd.read_csv(file_path)
    print(df.head())
    print(df.shape)
    return df

def forecast_accuracy (forecast, actual):
    mape = np.mean(np.abs((actual - forecast)/np.abs(actual))) # MAPE
    mae = np.mean(np.abs(actual- forecast)) # MAE
    rmse = np.mean((actual- forecast)**2)**.5 # RMSE
    corr = np.corrcoef(forecast, actual)[0,1] #corr
    return mape, mae, rmse, corr

In [4]:
file_path = "../Coding Challenge - Data(20221128)/stores.csv"
stores_df = file_read(file_path)

file_path = "../Coding Challenge - Data(20221128)/transactions.csv"
transactions_df = file_read(file_path)

file_path = "../Coding Challenge - Data(20221128)/users.csv"
users_df = file_read(file_path)

                                     id        nam       laa category   lat  \
0  139a9a4b-1cb4-cb91-f718-d0bdd7db31c9  Aichi Ken  Agui Cho   ショッピング 34.95   
1  437e0dae-dbf7-b1a8-4a03-010b9597bbd2  Aichi Ken  Agui Cho    マッサージ 34.95   
2  2deb0ff0-5909-c169-4a23-4ed3979467fc  Aichi Ken  Agui Cho      その他 34.93   
3  093585a3-96ce-7513-4d60-1dd36c4b30d2  Aichi Ken  Agui Cho      カフェ 34.95   
4  d0637fde-e4c8-4c60-5817-a3494a623a6f  Aichi Ken  Agui Cho      グルメ 34.92   

     lon  
0 136.94  
1 136.88  
2 136.93  
3 136.89  
4 136.89  
(99991, 6)
                                     id                               user_id  \
0  a555eb55-1da4-6087-b5ab-06b35b608002  93098549-3ff0-e579-01c3-df9183278f64   
1  f699b867-e70c-5160-90c8-7dfa32dce0da  35ba9da1-98d0-cbb7-7844-e648601b7208   
2  5c3f8abc-f4e6-94fd-7b13-fe0efd4e4591  63cb5ee0-a457-9975-5455-4d76109bc754   
3  f033da1a-bafb-5faa-67e3-c76b07b04333  786fcb82-40cf-1dc9-b9ba-2541502ca95a   
4  ccc35ffe-ee42-b696-783c-6670bfd800ec  ac

In [5]:
# Merged transactions with user and store data

transaction_data = (
    transactions_df
    .merge(users_df.rename(columns={'id': 'user_id'}), on='user_id', how='left')
    .merge(stores_df.rename(columns={'id': 'store_id'}), on='store_id', how='left')
)
print("\nMerged Data Sample:")
print(transaction_data.head())


Merged Data Sample:
                                     id                               user_id  \
0  a555eb55-1da4-6087-b5ab-06b35b608002  93098549-3ff0-e579-01c3-df9183278f64   
1  f699b867-e70c-5160-90c8-7dfa32dce0da  35ba9da1-98d0-cbb7-7844-e648601b7208   
2  5c3f8abc-f4e6-94fd-7b13-fe0efd4e4591  63cb5ee0-a457-9975-5455-4d76109bc754   
3  f033da1a-bafb-5faa-67e3-c76b07b04333  786fcb82-40cf-1dc9-b9ba-2541502ca95a   
4  ccc35ffe-ee42-b696-783c-6670bfd800ec  acc814f5-beb6-db35-64f1-52f7c5a8466c   

                               store_id            event_occurrence  amount  \
0  a3c7874b-75df-9cf2-62ab-cf58be5d9e0e  2020-01-01 06:40:43.323987     315   
1  1d59adf3-94f2-f5f8-46c6-8558c544a3a9  2020-01-01 07:04:25.968615    1646   
2  c835f09d-ac5b-2005-1959-f452456a746a  2020-01-01 07:06:16.164563     385   
3  eaf4c05d-13e1-7c8a-9ae0-cf2ec2e3dcd2  2020-01-01 07:07:26.029685     166   
4  a8301ae2-df3b-ffe4-0279-c74af1dbb847  2020-01-01 07:12:24.448015     656   

  gender   age   

In [10]:
transaction_data['event_occurrence'] = pd.to_datetime(transaction_data['event_occurrence'])
daily_gmv_v1 = transaction_data.groupby(transaction_data['event_occurrence'].dt.date)['amount'].sum().reset_index()
daily_gmv_v1.columns = ['ds', 'y']  # Renamed columns to be compatible with Prophet
daily_gmv_v1['ds'] = pd.to_datetime(daily_gmv_v1['ds'])
daily_gmv_v1.head()

,ds,y
0,2020-01-01,2633813
1,2020-01-02,2115625
2,2020-01-03,2110395
3,2020-01-04,2415564
4,2020-01-05,2346069


In [11]:
# Defined start and end dates
start_date = '2020-01-01'
end_date = '2022-01-31'

date_range = pd.date_range(start=start_date, end=end_date)
df = pd.DataFrame(date_range, columns=['ds'])
print(df)

            ds
0   2020-01-01
1   2020-01-02
2   2020-01-03
3   2020-01-04
4   2020-01-05
..         ...
757 2022-01-27
758 2022-01-28
759 2022-01-29
760 2022-01-30
761 2022-01-31

[762 rows x 1 columns]


In [12]:
daily_gmv = df.merge(daily_gmv_v1, how = 'left', on= 'ds')
daily_gmv['y'] = daily_gmv['y'].fillna(0)
daily_gmv.tail()

,ds,y
757,2022-01-27,0.00
758,2022-01-28,0.00
759,2022-01-29,0.00
760,2022-01-30,0.00
761,2022-01-31,0.00


In [13]:
daily_gmv['month'] = daily_gmv['ds'].dt.month
daily_gmv['date'] = daily_gmv['ds'].dt.day
daily_gmv['weekday'] = daily_gmv['ds'].dt.weekday
daily_gmv['weekend_flag'] = daily_gmv['weekday'].apply(lambda x: 1 if x in [5,6] else 0)
daily_gmv.head(20)

,ds,y,month,date,weekday,weekend_flag
0,2020-01-01,2633813.00,1,1,2,0
1,2020-01-02,2115625.00,1,2,3,0
2,2020-01-03,2110395.00,1,3,4,0
3,2020-01-04,2415564.00,1,4,5,1
4,2020-01-05,2346069.00,1,5,6,1
5,2020-01-06,2328688.00,1,6,0,0
6,2020-01-07,2148892.00,1,7,1,0
7,2020-01-08,2613571.00,1,8,2,0
8,2020-01-09,2346278.00,1,9,3,0
9,2020-01-10,2193921.00,1,10,4,0


In [14]:
# convert day of week & month to cyclical features

daily_gmv['day_of_week_sin'] = np.sin(2 * np.pi * daily_gmv['weekday'] / 7)
daily_gmv['day_of_week_cos'] = np.cos(2 * np.pi * daily_gmv['weekday'] / 7)


daily_gmv['month_sin'] = np.sin(2 * np.pi * daily_gmv['month'] / 12)
daily_gmv['month_cos'] = np.cos(2 * np.pi * daily_gmv['month'] / 12)
daily_gmv.head()

,ds,y,month,date,weekday,weekend_flag,day_of_week_sin,day_of_week_cos,month_sin,month_cos
0,2020-01-01,2633813.00,1,1,2,0,0.97,-0.22,0.50,0.87
1,2020-01-02,2115625.00,1,2,3,0,0.43,-0.90,0.50,0.87
2,2020-01-03,2110395.00,1,3,4,0,-0.43,-0.90,0.50,0.87
3,2020-01-04,2415564.00,1,4,5,1,-0.97,-0.22,0.50,0.87
4,2020-01-05,2346069.00,1,5,6,1,-0.78,0.62,0.50,0.87


In [15]:
# Step 1: Calculate lagged GMV (previous month's spend) for each user
daily_gmv['lagged_gmv_30'] = daily_gmv['y'].shift(30)
daily_gmv['lagged_gmv_30'] = daily_gmv['lagged_gmv_30'].fillna(0)
daily_gmv['lagged_gmv_60'] = daily_gmv['y'].shift(60)
daily_gmv['lagged_gmv_60'] = daily_gmv['lagged_gmv_60'].fillna(0)
daily_gmv['lagged_gmv_90'] = daily_gmv['y'].shift(90)
daily_gmv['lagged_gmv_90'] = daily_gmv['lagged_gmv_90'].fillna(0)
daily_gmv['lagged_gmv_120'] = daily_gmv['y'].shift(120)
daily_gmv['lagged_gmv_120'] = daily_gmv['lagged_gmv_120'].fillna(0)
daily_gmv['lagged_gmv_180'] = daily_gmv['y'].shift(180)
daily_gmv['lagged_gmv_180'] = daily_gmv['lagged_gmv_180'].fillna(0)
daily_gmv['lagged_gmv_365'] = daily_gmv['y'].shift(365)
daily_gmv['lagged_gmv_365'] = daily_gmv['lagged_gmv_365'].fillna(0)

daily_gmv.head(16)

,ds,y,month,date,weekday,weekend_flag,day_of_week_sin,day_of_week_cos,month_sin,month_cos,lagged_gmv_30,lagged_gmv_60,lagged_gmv_90,lagged_gmv_120,lagged_gmv_180,lagged_gmv_365
0,2020-01-01,2633813.00,1,1,2,0,0.97,-0.22,0.50,0.87,0.00,0.00,0.00,0.00,0.00,0.00
1,2020-01-02,2115625.00,1,2,3,0,0.43,-0.90,0.50,0.87,0.00,0.00,0.00,0.00,0.00,0.00
2,2020-01-03,2110395.00,1,3,4,0,-0.43,-0.90,0.50,0.87,0.00,0.00,0.00,0.00,0.00,0.00
3,2020-01-04,2415564.00,1,4,5,1,-0.97,-0.22,0.50,0.87,0.00,0.00,0.00,0.00,0.00,0.00
4,2020-01-05,2346069.00,1,5,6,1,-0.78,0.62,0.50,0.87,0.00,0.00,0.00,0.00,0.00,0.00
5,2020-01-06,2328688.00,1,6,0,0,0.00,1.00,0.50,0.87,0.00,0.00,0.00,0.00,0.00,0.00
6,2020-01-07,2148892.00,1,7,1,0,0.78,0.62,0.50,0.87,0.00,0.00,0.00,0.00,0.00,0.00
7,2020-01-08,2613571.00,1,8,2,0,0.97,-0.22,0.50,0.87,0.00,0.00,0.00,0.00,0.00,0.00
8,2020-01-09,2346278.00,1,9,3,0,0.43,-0.90,0.50,0.87,0.00,0.00,0.00,0.00,0.00,0.00
9,2020-01-10,2193921.00,1,10,4,0,-0.43,-0.90,0.50,0.87,0.00,0.00,0.00,0.00,0.00,0.00


In [16]:
daily_gmv['7_day_avg'] = daily_gmv['y'].rolling(window=7).mean()
daily_gmv['14_day_avg'] = daily_gmv['y'].rolling(window=14).mean()
daily_gmv['21_day_avg'] = daily_gmv['y'].rolling(window=21).mean()
daily_gmv['28_day_avg'] = daily_gmv['y'].rolling(window=28).mean()
daily_gmv['30_day_avg'] = daily_gmv['y'].rolling(window=30).mean()
daily_gmv['60_day_avg'] = daily_gmv['y'].rolling(window=60).mean()

daily_gmv['7_day_avg'] = daily_gmv['7_day_avg'].fillna(0)
daily_gmv['14_day_avg'] = daily_gmv['14_day_avg'].fillna(0)
daily_gmv['21_day_avg'] = daily_gmv['21_day_avg'].fillna(0)
daily_gmv['28_day_avg'] = daily_gmv['28_day_avg'].fillna(0)
daily_gmv['30_day_avg'] = daily_gmv['30_day_avg'].fillna(0)
daily_gmv['60_day_avg'] = daily_gmv['60_day_avg'].fillna(0)


daily_gmv['Lagged_30d_avg_7d'] = daily_gmv['7_day_avg'].shift(30)
daily_gmv['Lagged_30d_avg_14d'] = daily_gmv['14_day_avg'].shift(30)
daily_gmv['Lagged_30d_avg_21d'] = daily_gmv['21_day_avg'].shift(30)
daily_gmv['Lagged_30d_avg_28d'] = daily_gmv['28_day_avg'].shift(30)
daily_gmv['Lagged_30d_avg_30d'] = daily_gmv['30_day_avg'].shift(30)
daily_gmv['Lagged_30d_avg_60d'] = daily_gmv['60_day_avg'].shift(30)

daily_gmv['Lagged_30d_avg_7d'] = daily_gmv['Lagged_30d_avg_7d'].fillna(0)
daily_gmv['Lagged_30d_avg_14d'] = daily_gmv['Lagged_30d_avg_14d'].fillna(0)
daily_gmv['Lagged_30d_avg_21d'] = daily_gmv['Lagged_30d_avg_21d'].fillna(0)
daily_gmv['Lagged_30d_avg_28d'] = daily_gmv['Lagged_30d_avg_28d'].fillna(0)
daily_gmv['Lagged_30d_avg_30d'] = daily_gmv['Lagged_30d_avg_30d'].fillna(0)
daily_gmv['Lagged_30d_avg_60d'] = daily_gmv['Lagged_30d_avg_60d'].fillna(0)
daily_gmv.head(31)

,ds,y,month,date,weekday,weekend_flag,day_of_week_sin,day_of_week_cos,month_sin,month_cos,lagged_gmv_30,lagged_gmv_60,lagged_gmv_90,lagged_gmv_120,lagged_gmv_180,lagged_gmv_365,7_day_avg,14_day_avg,21_day_avg,28_day_avg,30_day_avg,60_day_avg,Lagged_30d_avg_7d,Lagged_30d_avg_14d,Lagged_30d_avg_21d,Lagged_30d_avg_28d,Lagged_30d_avg_30d,Lagged_30d_avg_60d
0,2020-01-01,2633813.00,1,1,2,0,0.97,-0.22,0.50,0.87,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
1,2020-01-02,2115625.00,1,2,3,0,0.43,-0.90,0.50,0.87,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
2,2020-01-03,2110395.00,1,3,4,0,-0.43,-0.90,0.50,0.87,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
3,2020-01-04,2415564.00,1,4,5,1,-0.97,-0.22,0.50,0.87,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
4,2020-01-05,2346069.00,1,5,6,1,-0.78,0.62,0.50,0.87,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
5,2020-01-06,2328688.00,1,6,0,0,0.00,1.00,0.50,0.87,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
6,2020-01-07,2148892.00,1,7,1,0,0.78,0.62,0.50,0.87,0.00,0.00,0.00,0.00,0.00,0.00,2299863.71,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
7,2020-01-08,2613571.00,1,8,2,0,0.97,-0.22,0.50,0.87,0.00,0.00,0.00,0.00,0.00,0.00,2296972.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
8,2020-01-09,2346278.00,1,9,3,0,0.43,-0.90,0.50,0.87,0.00,0.00,0.00,0.00,0.00,0.00,2329922.43,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
9,2020-01-10,2193921.00,1,10,4,0,-0.43,-0.90,0.50,0.87,0.00,0.00,0.00,0.00,0.00,0.00,2341854.71,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


# Gender wise transactions variables

In [17]:
txn_gen_summary = transaction_data.groupby(transaction_data['event_occurrence'].dt.date).agg(
    male_spend = ('amount', lambda x: x[transaction_data['gender']== 'M'].sum()),
    female_spend = ('amount', lambda x: x[transaction_data['gender']== 'F'].sum())
).reset_index()
txn_gen_summary.head()

,event_occurrence,male_spend,female_spend
0,2020-01-01,1354948,1119791
1,2020-01-02,1151790,789462
2,2020-01-03,1182202,790757
3,2020-01-04,1257838,865342
4,2020-01-05,1217661,926551


In [18]:
txn_gen_summary['Lagged_30d_male_spend'] = txn_gen_summary['male_spend'].shift(30)
txn_gen_summary['Lagged_30d_female_spend'] = txn_gen_summary['female_spend'].shift(30)



txn_gen_summary.head()

,event_occurrence,male_spend,female_spend,Lagged_30d_male_spend,Lagged_30d_female_spend
0,2020-01-01,1354948,1119791,0.00,0.00
1,2020-01-02,1151790,789462,0.00,0.00
2,2020-01-03,1182202,790757,0.00,0.00
3,2020-01-04,1257838,865342,0.00,0.00
4,2020-01-05,1217661,926551,0.00,0.00


# Category wise transactions variables

In [19]:
# Translated the transaction category from Japaneese to English 
translator = GoogleTranslator(source='ja', target='en')

unique_values = transaction_data['category'].unique()
translations = {value: translator.translate(value) for value in unique_values}

transaction_data['category_english'] = transaction_data['category'].map(translations)
print(transaction_data[['category', 'category_english']].head())

    category   category_english
0       コンビニ  Convenience store
1     ショッピング           shopping
2       コンビニ  Convenience store
3        グルメ            Gourmet
4  スーパーマーケット        supermarket


In [20]:
# Identified top transacted category

result = transaction_data.groupby ('category_english').agg(Total_amount = ('amount','sum')).reset_index()
Total_amount = result['Total_amount'].sum()
result['Percentage'] = (result['Total_amount']/Total_amount) *100

result_sorted = result.sort_values(by = 'Total_amount', ascending= False)

print(result_sorted)

                          category_english  Total_amount  Percentage
14                                shopping     576220613       21.37
1                        Convenience store     337061594       12.50
5                                  Gourmet     331995327       12.31
15                             supermarket     306594892       11.37
11                                 fashion     240102416        8.91
13                                  others     158691578        5.89
0                                     Cafe     140278627        5.20
7        Home appliances and mobile phones     128323273        4.76
6                             Hairdressers     111032458        4.12
12                                 massage      69733513        2.59
3                Drugstores and pharmacies      68269640        2.53
4                            Entertainment      67328201        2.50
9                        Hotels and Ryokan      61128702        2.27
8                    Hospitals and

In [21]:
txn_cat_summary = transaction_data.groupby(transaction_data['event_occurrence'].dt.date).agg(
    shopping = ('amount', lambda x: x[transaction_data['category_english']== 'Shopping'].sum()),
    comvenience_store = ('amount', lambda x: x[transaction_data['category_english']== 'Convenience store'].sum()),
    gourmet = ('amount', lambda x: x[transaction_data['category_english']== 'Gourmet'].sum()),    
    supermarket = ('amount', lambda x: x[transaction_data['category_english']== 'supermarket'].sum()),    
    fashion = ('amount', lambda x: x[transaction_data['category_english']== 'fashion'].sum()),    
    # others = ('amount', lambda x: x[transaction_data['category_english']== 'others'].sum()),    
    # cafe = ('amount', lambda x: x[transaction_data['category_english']== 'Cafe'].sum()),    
    # appliance = ('amount', lambda x: x[transaction_data['category_english']== 'Home appliances and mobile phones'].sum()),    
    # hairdressers = ('amount', lambda x: x[transaction_data['category_english']== 'Hairdressers'].sum()),    
    # pharmacy = ('amount', lambda x: x[transaction_data['category_english']== 'Drugstores and pharmacies'].sum()),    
    # entertainment = ('amount', lambda x: x[transaction_data['category_english']== 'Entertainment'].sum()),  
    # hotel = ('amount', lambda x: x[transaction_data['category_english']== 'Hotels and Ryokan'].sum()),  
    # hospital = ('amount', lambda x: x[transaction_data['category_english']== 'Hospitals and Clinics'].sum()),  
    # department_store = ('amount', lambda x: x[transaction_data['category_english']== 'Department stores and shopping centers'].sum()),  
    # cleaning = ('amount', lambda x: x[transaction_data['category_english']== 'cleaning'].sum())
).reset_index()
txn_cat_summary.head()

,event_occurrence,shopping,comvenience_store,gourmet,supermarket,fashion
0,2020-01-01,0,382419,290863,292735,132525
1,2020-01-02,0,282691,317518,216898,198231
2,2020-01-03,0,277644,202788,286903,200913
3,2020-01-04,0,305970,259361,276585,181409
4,2020-01-05,0,300589,337682,265383,219227


In [22]:
txn_cat_summary['Lagged_30d_shopping'] = txn_cat_summary['shopping'].shift(30)
txn_cat_summary['Lagged_30d_comvenience_store'] = txn_cat_summary['comvenience_store'].shift(30)
txn_cat_summary['Lagged_30d_gourmet'] = txn_cat_summary['gourmet'].shift(30)
txn_cat_summary['Lagged_30d_supermarket'] = txn_cat_summary['supermarket'].shift(30)
txn_cat_summary['Lagged_30d_fashion'] = txn_cat_summary['fashion'].shift(30)
# txn_cat_summary['Lagged_30d_others'] = txn_cat_summary['others'].shift(30)
# txn_cat_summary['Lagged_30d_cafe'] = txn_cat_summary['cafe'].shift(30)
# txn_cat_summary['Lagged_30d_appliance'] = txn_cat_summary['appliance'].shift(30)
# txn_cat_summary['Lagged_30d_hairdressers'] = txn_cat_summary['hairdressers'].shift(30)
# txn_cat_summary['Lagged_30d_pharmacy'] = txn_cat_summary['pharmacy'].shift(30)
# txn_cat_summary['Lagged_30d_entertainment'] = txn_cat_summary['entertainment'].shift(30)
# txn_cat_summary['Lagged_30d_hotel'] = txn_cat_summary['hotel'].shift(30)
# txn_cat_summary['Lagged_30d_hospital'] = txn_cat_summary['hospital'].shift(30)
# txn_cat_summary['Lagged_30d_department_store'] = txn_cat_summary['department_store'].shift(30)
# txn_cat_summary['Lagged_30d_cleaning'] = txn_cat_summary['cleaning'].shift(30)



txn_cat_summary.head()

,event_occurrence,shopping,comvenience_store,gourmet,supermarket,fashion,Lagged_30d_shopping,Lagged_30d_comvenience_store,Lagged_30d_gourmet,Lagged_30d_supermarket,Lagged_30d_fashion
0,2020-01-01,0,382419,290863,292735,132525,0.00,0.00,0.00,0.00,0.00
1,2020-01-02,0,282691,317518,216898,198231,0.00,0.00,0.00,0.00,0.00
2,2020-01-03,0,277644,202788,286903,200913,0.00,0.00,0.00,0.00,0.00
3,2020-01-04,0,305970,259361,276585,181409,0.00,0.00,0.00,0.00,0.00
4,2020-01-05,0,300589,337682,265383,219227,0.00,0.00,0.00,0.00,0.00


In [23]:
txn_gen_summary['event_occurrence'] = pd.to_datetime(txn_gen_summary['event_occurrence'])
txn_cat_summary['event_occurrence'] = pd.to_datetime(txn_cat_summary['event_occurrence'])

In [24]:

daily_gmv = (
    daily_gmv
    .merge(txn_gen_summary.rename(columns={'event_occurrence': 'ds'}), on='ds', how='left')
    .merge(txn_cat_summary.rename(columns={'event_occurrence': 'ds'}), on='ds', how='left')
)
daily_gmv.head()


,ds,y,month,date,weekday,weekend_flag,day_of_week_sin,day_of_week_cos,month_sin,month_cos,lagged_gmv_30,lagged_gmv_60,lagged_gmv_90,lagged_gmv_120,lagged_gmv_180,lagged_gmv_365,7_day_avg,14_day_avg,21_day_avg,28_day_avg,30_day_avg,60_day_avg,Lagged_30d_avg_7d,Lagged_30d_avg_14d,Lagged_30d_avg_21d,Lagged_30d_avg_28d,Lagged_30d_avg_30d,Lagged_30d_avg_60d,male_spend,female_spend,Lagged_30d_male_spend,Lagged_30d_female_spend,shopping,comvenience_store,gourmet,supermarket,fashion,Lagged_30d_shopping,Lagged_30d_comvenience_store,Lagged_30d_gourmet,Lagged_30d_supermarket,Lagged_30d_fashion
0,2020-01-01,2633813.00,1,1,2,0,0.97,-0.22,0.50,0.87,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1354948.00,1119791.00,0.00,0.00,0.00,382419.00,290863.00,292735.00,132525.00,0.00,0.00,0.00,0.00,0.00
1,2020-01-02,2115625.00,1,2,3,0,0.43,-0.90,0.50,0.87,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1151790.00,789462.00,0.00,0.00,0.00,282691.00,317518.00,216898.00,198231.00,0.00,0.00,0.00,0.00,0.00
2,2020-01-03,2110395.00,1,3,4,0,-0.43,-0.90,0.50,0.87,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1182202.00,790757.00,0.00,0.00,0.00,277644.00,202788.00,286903.00,200913.00,0.00,0.00,0.00,0.00,0.00
3,2020-01-04,2415564.00,1,4,5,1,-0.97,-0.22,0.50,0.87,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1257838.00,865342.00,0.00,0.00,0.00,305970.00,259361.00,276585.00,181409.00,0.00,0.00,0.00,0.00,0.00
4,2020-01-05,2346069.00,1,5,6,1,-0.78,0.62,0.50,0.87,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1217661.00,926551.00,0.00,0.00,0.00,300589.00,337682.00,265383.00,219227.00,0.00,0.00,0.00,0.00,0.00


In [34]:
daily_gmv['Lagged_30d_male_spend'] = daily_gmv['Lagged_30d_male_spend'].fillna(0)
daily_gmv['Lagged_30d_female_spend'] = daily_gmv['Lagged_30d_female_spend'].fillna(0)

daily_gmv['Lagged_30d_shopping']=daily_gmv['Lagged_30d_shopping'].fillna(0)
daily_gmv['Lagged_30d_comvenience_store']=daily_gmv['Lagged_30d_comvenience_store'].fillna(0)
daily_gmv['Lagged_30d_gourmet']=daily_gmv['Lagged_30d_gourmet'].fillna(0)
daily_gmv['Lagged_30d_supermarket']=daily_gmv['Lagged_30d_supermarket'].fillna(0)
daily_gmv['Lagged_30d_fashion']=daily_gmv['Lagged_30d_fashion'].fillna(0)


In [25]:
daily_gmv.tail(31)

,ds,y,month,date,weekday,weekend_flag,day_of_week_sin,day_of_week_cos,month_sin,month_cos,lagged_gmv_30,lagged_gmv_60,lagged_gmv_90,lagged_gmv_120,lagged_gmv_180,lagged_gmv_365,7_day_avg,14_day_avg,21_day_avg,28_day_avg,30_day_avg,60_day_avg,Lagged_30d_avg_7d,Lagged_30d_avg_14d,Lagged_30d_avg_21d,Lagged_30d_avg_28d,Lagged_30d_avg_30d,Lagged_30d_avg_60d,male_spend,female_spend,Lagged_30d_male_spend,Lagged_30d_female_spend,shopping,comvenience_store,gourmet,supermarket,fashion,Lagged_30d_shopping,Lagged_30d_comvenience_store,Lagged_30d_gourmet,Lagged_30d_supermarket,Lagged_30d_fashion
731,2022-01-01,0.00,1,1,5,1,-0.97,-0.22,0.50,0.87,5858878.00,5524569.00,5109116.00,4522859.00,4252801.00,4111455.00,5407523.29,5842508.79,5939502.67,5950717.39,5956040.77,5936965.22,6093405.43,6056433.50,5955230.90,5892347.71,5917889.67,5772414.42,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
732,2022-01-02,0.00,1,2,6,1,-0.78,0.62,0.50,0.87,5642807.00,6949776.00,5424723.00,5662368.00,4487366.00,3741056.00,4412701.57,5319783.64,5624534.86,5729179.04,5767947.20,5821135.62,6011331.71,6074029.86,5974895.24,5904443.75,5874324.03,5776049.15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
733,2022-01-03,0.00,1,3,0,0,0.00,1.00,0.50,0.87,6418329.00,5601178.00,5290304.00,5734441.00,4395646.00,3706206.00,3563545.71,4903504.00,5350132.62,5522058.46,5554002.90,5727782.65,5934091.86,6094486.86,6000144.19,5921892.14,5901562.40,5794849.57,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
734,2022-01-04,0.00,1,4,1,0,0.78,0.62,0.50,0.87,6203074.00,5304118.00,5250033.00,4774223.00,4359944.00,3494929.00,2671298.71,4467555.07,5058926.67,5318097.68,5347233.77,5639380.68,5893829.57,6079694.00,5999627.52,5943660.46,5931527.60,5810733.58,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
735,2022-01-05,0.00,1,5,2,0,0.97,-0.22,0.50,0.87,5799376.00,5929774.00,4918469.00,4868274.00,4658080.00,3451947.00,1748117.71,4035012.29,4782064.95,5107501.07,5153921.23,5540551.12,5928710.00,6097397.50,6014577.24,5947745.32,5927181.00,5825415.37,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
736,2022-01-06,0.00,1,6,3,0,0.43,-0.90,0.50,0.87,5710902.00,5593561.00,4995602.00,4235975.00,4760855.00,3480360.00,879107.29,3607700.21,4491109.90,4912804.82,4963557.83,5447325.10,5904678.57,6011862.21,6022413.33,5950618.96,5931092.37,5837337.03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
737,2022-01-07,0.00,1,7,4,0,-0.43,-0.90,0.50,0.87,5896705.00,5685000.00,5791723.00,4742142.00,4618887.00,3069965.00,0.00,3168597.00,4221163.62,4702007.21,4767001.00,5352575.10,5932867.29,6011170.64,6025387.81,5932619.96,5938149.20,5839086.73,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
738,2022-01-08,0.00,1,8,5,1,-0.97,-0.22,0.50,0.87,5451495.00,5630440.00,6053856.00,4601142.00,4899724.00,3230225.00,0.00,2703761.64,3895005.86,4454627.00,4585284.50,5258734.43,5874669.71,5984037.57,5995845.57,5935090.61,5932184.37,5829047.38,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
739,2022-01-09,0.00,1,9,6,1,-0.78,0.62,0.50,0.87,5902333.00,6400677.00,5426186.00,5491014.00,4337525.00,3673395.00,0.00,2206350.79,3546522.43,4218401.14,4388540.07,5152056.48,5911744.86,5961538.29,6019934.86,5959107.64,5915572.90,5836983.17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
740,2022-01-10,0.00,1,10,0,0,0.00,1.00,0.50,0.87,6926646.00,5382317.00,5260981.00,6858149.00,4155670.00,3525379.00,0.00,1781772.86,3269002.67,4012599.46,4157651.87,5062351.20,5984361.57,5959226.71,6057778.43,5996198.54,5967050.53,5864744.25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [31]:
daily_gmv.columns

Index(['ds', 'y', 'month', 'date', 'weekday', 'weekend_flag',
       'day_of_week_sin', 'day_of_week_cos', 'month_sin', 'month_cos',
       'lagged_gmv_30', 'lagged_gmv_60', 'lagged_gmv_90', 'lagged_gmv_120',
       'lagged_gmv_180', 'lagged_gmv_365', '7_day_avg', '14_day_avg',
       '21_day_avg', '28_day_avg', '30_day_avg', '60_day_avg',
       'Lagged_30d_avg_7d', 'Lagged_30d_avg_14d', 'Lagged_30d_avg_21d',
       'Lagged_30d_avg_28d', 'Lagged_30d_avg_30d', 'Lagged_30d_avg_60d',
       'male_spend', 'female_spend', 'Lagged_30d_male_spend',
       'Lagged_30d_female_spend', 'shopping', 'comvenience_store', 'gourmet',
       'supermarket', 'fashion', 'Lagged_30d_shopping',
       'Lagged_30d_comvenience_store', 'Lagged_30d_gourmet',
       'Lagged_30d_supermarket', 'Lagged_30d_fashion'],
      dtype='object')

In [35]:
daily_gmv_full = daily_gmv

daily_gmv = daily_gmv.drop([ '7_day_avg', '14_day_avg','21_day_avg', '28_day_avg', '30_day_avg', '60_day_avg', 'male_spend', 'female_spend',  'shopping', 'comvenience_store', 'gourmet', 'supermarket', 'fashion'], axis= 1)

KeyError: "['7_day_avg', '14_day_avg', '21_day_avg', '28_day_avg', '30_day_avg', '60_day_avg', 'male_spend', 'female_spend', 'shopping', 'comvenience_store', 'gourmet', 'supermarket', 'fashion'] not found in axis"

In [36]:
daily_gmv.tail()

,ds,y,month,date,weekday,weekend_flag,day_of_week_sin,day_of_week_cos,month_sin,month_cos,lagged_gmv_30,lagged_gmv_60,lagged_gmv_90,lagged_gmv_120,lagged_gmv_180,lagged_gmv_365,Lagged_30d_avg_7d,Lagged_30d_avg_14d,Lagged_30d_avg_21d,Lagged_30d_avg_28d,Lagged_30d_avg_30d,Lagged_30d_avg_60d,Lagged_30d_male_spend,Lagged_30d_female_spend,Lagged_30d_shopping,Lagged_30d_comvenience_store,Lagged_30d_gourmet,Lagged_30d_supermarket,Lagged_30d_fashion
757,2022-01-27,0.00,1,27,3,0,0.43,-0.90,0.50,0.87,6245729.00,6484910.00,5272068.00,5552970.00,5155196.00,3899875.00,6263811.43,6252740.64,6200364.00,6126442.64,6099157.63,6028330.72,0.00,0.00,0.00,0.00,0.00,0.00,0.00
758,2022-01-28,0.00,1,28,4,0,-0.43,-0.90,0.50,0.87,6462267.00,5555213.00,6352360.00,5454223.00,5218813.00,3649588.00,6321906.86,6299038.57,6227295.52,6153688.46,6129392.77,6030162.50,0.00,0.00,0.00,0.00,0.00,0.00,0.00
759,2022-01-29,0.00,1,29,5,1,-0.97,-0.22,0.50,0.87,6083073.00,5879122.00,6766815.00,5269000.00,4320924.00,3585275.00,6336293.14,6297111.21,6257370.67,6161695.43,6136191.13,6018766.80,0.00,0.00,0.00,0.00,0.00,0.00,0.00
760,2022-01-30,0.00,1,30,6,1,-0.78,0.62,0.50,0.87,6153751.00,5699384.00,5537277.00,5737172.00,4500197.00,3808085.00,6337194.00,6331745.43,6269342.95,6179943.43,6151336.70,6029041.37,0.00,0.00,0.00,0.00,0.00,0.00,0.00
761,2022-01-31,0.00,1,31,0,0,0.00,1.00,0.50,0.87,0.00,5858878.00,5524569.00,5109116.00,4581516.00,3884907.00,5407523.29,5842508.79,5939502.67,5950717.39,5956040.77,5936965.22,0.00,0.00,0.00,0.00,0.00,0.00,0.00


In [37]:
# Train-test split: Jan 2020 - Sep 2021 for training, Oct - Dec 2021 for testing
train_data = daily_gmv[(daily_gmv['ds'] >= '2020-01-01') & (daily_gmv['ds'] <= '2021-11-30')]
test_data = daily_gmv[(daily_gmv['ds'] >= '2021-12-01') & (daily_gmv['ds'] <= '2021-12-31')]
oot_data = daily_gmv[(daily_gmv['ds'] >= '2022-01-01') & (daily_gmv['ds'] <= '2022-01-31')]

In [38]:
comb_list = test_data.columns.to_list()
del comb_list[:2]
comb_list

['month',
 'date',
 'weekday',
 'weekend_flag',
 'day_of_week_sin',
 'day_of_week_cos',
 'month_sin',
 'month_cos',
 'lagged_gmv_30',
 'lagged_gmv_60',
 'lagged_gmv_90',
 'lagged_gmv_120',
 'lagged_gmv_180',
 'lagged_gmv_365',
 'Lagged_30d_avg_7d',
 'Lagged_30d_avg_14d',
 'Lagged_30d_avg_21d',
 'Lagged_30d_avg_28d',
 'Lagged_30d_avg_30d',
 'Lagged_30d_avg_60d',
 'Lagged_30d_male_spend',
 'Lagged_30d_female_spend',
 'Lagged_30d_shopping',
 'Lagged_30d_comvenience_store',
 'Lagged_30d_gourmet',
 'Lagged_30d_supermarket',
 'Lagged_30d_fashion']

In [67]:
# Single variable regressor

def mape_comp(comb_list, excel_name):
    result = []
    var_list = []
    p=0
    for idx, i in enumerate(comb_list):
        model = Prophet(daily_seasonality= True, weekly_seasonality= True, yearly_seasonality = True, holidays_mode = 'additive')
        model.add_country_holidays(country_name = 'JP')
        model.add_regressor('date')
        model.add_regressor('month_sin')   
        model.add_regressor('Lagged_30d_male_spend')
        model.add_regressor('Lagged_30d_avg_60d')
        model.add_regressor('Lagged_30d_fashion')
        model.add_regressor('month_cos')
        model.add_regressor('lagged_gmv_120')
        model.add_regressor('lagged_gmv_365')
        model.add_regressor('Lagged_30d_avg_21d')
        model.add_regressor(i)
        
        model.fit(train_data)
        p+=1
        print(p)
        
        prediction_train = model.predict(train_data)
        prediction_test = model.predict(test_data)
        
        prediction_df_train = train_data.merge(prediction_train[['ds','yhat']], on='ds', how= 'inner')
        prediction_df_test = test_data.merge(prediction_test[['ds','yhat']], on='ds', how= 'inner')
        
        var= list(i)
        Para = ('MAPE')
        
        train = forecast_accuracy(prediction_df_train['yhat'].values, train_data['y'].values)[0]
        test = forecast_accuracy(prediction_df_test['yhat'].values, test_data['y'].values)[0]
        
        result.append({'Index': idx, 'Para' : Para, 'Train': train, 'Test': test})
        var_list.append({'Index': idx, 'Variables': i})
        
    Result_df = pd.DataFrame(result)
    Var_list_df = pd.DataFrame(var_list)
    
    with pd.ExcelWriter(excel_name, engine = 'openpyxl') as writer:
        Result_df.to_excel(writer, sheet_name='Result', index = False, startrow = 0, startcol = 0)
        Var_list_df.to_excel(writer, sheet_name='Result', index = False, startrow = 0, startcol = 4)
    
    return(Result_df, Var_list_df)
        
        

In [68]:
mape_comp(comb_list,'../Coding_solution/Single_regressor_v10.xlsx')

23:16:16 - cmdstanpy - INFO - Chain [1] start processing
23:16:16 - cmdstanpy - INFO - Chain [1] done processing


1


23:16:17 - cmdstanpy - INFO - Chain [1] start processing
23:16:17 - cmdstanpy - INFO - Chain [1] done processing


2


23:16:17 - cmdstanpy - INFO - Chain [1] start processing
23:16:17 - cmdstanpy - INFO - Chain [1] done processing


...

23:16:27 - cmdstanpy - INFO - Chain [1] start processing
23:16:27 - cmdstanpy - INFO - Chain [1] done processing


26


23:16:27 - cmdstanpy - INFO - Chain [1] start processing
23:16:27 - cmdstanpy - INFO - Chain [1] done processing


27


(    Index  Para  Train   Test
 0       0  MAPE 0.0514 0.0328
 1       1  MAPE 0.0513 0.0318
 2       2  MAPE 0.0513 0.0318
 3       3  MAPE 0.0513 0.0318
 4       4  MAPE 0.0513 0.0318
 5       5  MAPE 0.0513 0.0318
 6       6  MAPE 0.0513 0.0318
 7       7  MAPE 0.0513 0.0318
 8       8  MAPE 0.0513 0.0318
 9       9  MAPE 0.0514 0.0319
 10     10  MAPE 0.0513 0.0323
 11     11  MAPE 0.0513 0.0318
 12     12  MAPE 0.0513 0.0325
 13     13  MAPE 0.0513 0.0318
 14     14  MAPE 0.0512 0.0320
 15     15  MAPE 0.0512 0.0318
 16     16  MAPE 0.0513 0.0318
 17     17  MAPE 0.0513 0.0318
 18     18  MAPE 0.0513 0.0315
 19     19  MAPE 0.0513 0.0318
 20     20  MAPE 0.0513 0.0318
 21     21  MAPE 0.0514 0.0316
 22     22  MAPE 0.0513 0.0318
 23     23  MAPE 0.0514 0.0325
 24     24  MAPE 0.0513 0.0320
 25     25  MAPE 0.0513 0.0318
 26     26  MAPE 0.0513 0.0318,
     Index                     Variables
 0       0                         month
 1       1                          date
 2      

In [71]:
model = Prophet(daily_seasonality= True, weekly_seasonality= True, yearly_seasonality = True, holidays_mode = 'additive')
model.add_country_holidays(country_name = 'JP')

model.add_regressor('date')
model.add_regressor('month_sin')   
model.add_regressor('Lagged_30d_male_spend')
model.add_regressor('Lagged_30d_avg_60d')
model.add_regressor('Lagged_30d_fashion')
model.add_regressor('month_cos')
model.add_regressor('lagged_gmv_120')
model.add_regressor('lagged_gmv_365')
model.add_regressor('Lagged_30d_avg_21d')
model.add_regressor('Lagged_30d_avg_30d')
model.fit(train_data)


prediction_train = model.predict(train_data)
prediction_test = model.predict(test_data)

prediction_df_train = train_data.merge(prediction_train[['ds','yhat']], on='ds', how= 'inner')
prediction_df_test = test_data.merge(prediction_test[['ds','yhat']], on='ds', how= 'inner')

Para = ('MAPE')

train = forecast_accuracy(prediction_df_train['yhat'].values, train_data['y'].values)[0]
test = forecast_accuracy(prediction_df_test['yhat'].values, test_data['y'].values)[0]


23:20:20 - cmdstanpy - INFO - Chain [1] start processing
23:20:20 - cmdstanpy - INFO - Chain [1] done processing


In [72]:
var_list = ['ds','date','month_sin','Lagged_30d_male_spend','Lagged_30d_avg_60d','Lagged_30d_fashion','month_cos','lagged_gmv_120','lagged_gmv_365','Lagged_30d_avg_21d','Lagged_30d_avg_30d']

In [73]:
# Forecast for the test period to evaluate accuracy
future_test = model.make_future_dataframe(periods=len(test_data), freq='D')
future_test= future_test.merge(daily_gmv[var_list], on='ds', how='left')
forecast_test = model.predict(future_test)

In [74]:
# Calculate RMSE on test data
test_forecast = forecast_test[['ds', 'yhat']].merge(test_data, on='ds', how='inner')
rmse = np.sqrt(mean_squared_error(test_forecast['y'], test_forecast['yhat']))
print(f"RMSE on Test Data: {rmse}")

# Calculate MAPE
mape = np.mean(np.abs((test_forecast['y'] - test_forecast['yhat']) / test_forecast['y'])) * 100
print(f"MAPE on Test Data: {mape:.2f}%")

RMSE on Test Data: 284004.9157999953
MAPE on Test Data: 3.15%


In [75]:
# Step 3: Forecast for January 2022
future_dates = model.make_future_dataframe(periods=62, freq='D', include_history=True)
future_dates

,ds
0,2020-01-01
1,2020-01-02
2,2020-01-03
3,2020-01-04
4,2020-01-05
...,...
757,2022-01-27
758,2022-01-28
759,2022-01-29
760,2022-01-30


In [76]:
future_dates= future_dates.merge(daily_gmv[list], on='ds', how='left')
future_dates
forecast_january = model.predict(future_dates)

In [77]:
# Filter out forecast for January 2022 only
forecast_january_2022 = forecast_january[(forecast_january['ds'] >= '2022-01-01') & (forecast_january['ds'] <= '2022-01-31')]

# Output results
print("Forecasted GMV for each day in January 2022:")
print(forecast_january_2022[['ds', 'yhat']])

Forecasted GMV for each day in January 2022:
            ds         yhat
731 2022-01-01 7282772.7621
732 2022-01-02 6643561.0755
733 2022-01-03 6340681.9648
734 2022-01-04 6284711.0713
735 2022-01-05 6339267.8542
736 2022-01-06 6377034.8979
737 2022-01-07 6318467.1528
738 2022-01-08 6706054.7686
739 2022-01-09 6743227.1323
740 2022-01-10 6996351.2393
741 2022-01-11 6428300.3980
742 2022-01-12 6420170.9249
743 2022-01-13 6499246.5229
744 2022-01-14 6444127.6716
745 2022-01-15 6798710.9723
746 2022-01-16 6829008.4802
747 2022-01-17 6533146.0948
748 2022-01-18 6471245.7585
749 2022-01-19 6532624.1961
750 2022-01-20 6526086.1325
751 2022-01-21 6500257.8974
752 2022-01-22 6863847.6697
753 2022-01-23 6900687.9267
754 2022-01-24 6595829.2631
755 2022-01-25 6477173.9336
756 2022-01-26 6558896.9075
757 2022-01-27 6587431.3475
758 2022-01-28 6542203.5410
759 2022-01-29 6902426.9326
760 2022-01-30 6929051.1305
761 2022-01-31 6675450.9665


In [78]:
forecast_january_2022

,ds,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,Autumnal Equinox,Autumnal Equinox_lower,Autumnal Equinox_upper,Children's Day,Children's Day_lower,Children's Day_upper,Coming of Age Day,Coming of Age Day_lower,Coming of Age Day_upper,Constitution Day,Constitution Day_lower,Constitution Day_upper,Culture Day,Culture Day_lower,Culture Day_upper,Emperor's Birthday,Emperor's Birthday_lower,Emperor's Birthday_upper,Foundation Day,Foundation Day_lower,Foundation Day_upper,Greenery Day,Greenery Day_lower,Greenery Day_upper,Labor Thanksgiving Day,Labor Thanksgiving Day_lower,Labor Thanksgiving Day_upper,Lagged_30d_avg_21d,Lagged_30d_avg_21d_lower,Lagged_30d_avg_21d_upper,Lagged_30d_avg_30d,Lagged_30d_avg_30d_lower,Lagged_30d_avg_30d_upper,Lagged_30d_avg_60d,Lagged_30d_avg_60d_lower,Lagged_30d_avg_60d_upper,Lagged_30d_fashion,Lagged_30d_fashion_lower,Lagged_30d_fashion_upper,Lagged_30d_male_spend,Lagged_30d_male_spend_lower,Lagged_30d_male_spend_upper,Marine Day,Marine Day_lower,Marine Day_upper,Mountain Day,Mountain Day_lower,Mountain Day_upper,New Year's Day,New Year's Day_lower,New Year's Day_upper,Respect for the Aged Day,Respect for the Aged Day_lower,Respect for the Aged Day_upper,Showa Day,Showa Day_lower,Showa Day_upper,Sports Day,Sports Day_lower,Sports Day_upper,Substitute Holiday,Substitute Holiday_lower,Substitute Holiday_upper,Vernal Equinox Day,Vernal Equinox Day_lower,Vernal Equinox Day_upper,additive_terms,additive_terms_lower,additive_terms_upper,daily,daily_lower,daily_upper,date,date_lower,date_upper,extra_regressors_additive,extra_regressors_additive_lower,extra_regressors_additive_upper,holidays,holidays_lower,holidays_upper,lagged_gmv_120,lagged_gmv_120_lower,lagged_gmv_120_upper,lagged_gmv_365,lagged_gmv_365_lower,lagged_gmv_365_upper,month_cos,month_cos_lower,month_cos_upper,month_sin,month_sin_lower,month_sin_upper,weekly,weekly_lower,weekly_upper,yearly,yearly_lower,yearly_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,yhat
731,2022-01-01,7183830.9149,6944677.8339,7625349.3923,7179576.5879,7188081.7787,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,-298135.0633,-298135.0633,-298135.0633,169030.2237,169030.2237,169030.2237,-212582.3478,-212582.3478,-212582.3478,-34904.6374,-34904.6374,-34904.6374,194271.3168,194271.3168,194271.3168,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,613385.7544,613385.7544,613385.7544,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,98941.8472,98941.8472,98941.8472,-762941.0995,-762941.0995,-762941.0995,-52869.8654,-52869.8654,-52869.8654,-385561.3038,-385561.3038,-385561.3038,613385.7544,613385.7544,613385.7544,-34022.0392,-34022.0392,-34022.0392,201876.5479,201876.5479,201876.5479,-186380.0634,-186380.0634,-186380.0634,-131845.3757,-131845.3757,-131845.3757,237961.2389,237961.2389,237961.2389,396097.2572,396097.2572,396097.2572,0.0000,0.0000,0.0000,7282772.7621
732,2022-01-02,7192480.1126,6304464.7231,6991995.3891,7187935.5335,7196943.2149,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,-300270.2801,-300270.2801,-300270.2801,166362.1452,166362.1452,166362.1452,-212860.0845,-212860.0845,-212860.0845,-34904.6374,-34904.6374,-34904.6374,194271.3168,194271.3168,194271.3168,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,-548919.0371,-548919.0371,-548919.0371,-762941.0995,-762941.0995,-762941.0995,-49279.9046,-49279.9046,-49279.9046,-433599.3359,-433599.3359,-433599.3359,0.0000,0.0000,0.0000,-54658.5605,-54658.5605,-54658.5605,175966.1084,175966.1084,175966.1084,-186380.0634,-186380.0634,-186380.063

In [79]:
forecast_january_2022.to_csv('../Coding_solution/daily_forecast_GMV_jan22_final_overall.csv')

In [80]:
forecast_january_2022[['ds', 'yhat']].to_csv('../Coding_solution/daily_forecast_GMV_jan22_final.csv')